In [ ]:
import time
import torch
import numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

import torch
import time
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from datasets import load_dataset



In [ ]:

# Load model and tokenizer
model_name = "gpt2"  # Replace with your LLM
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


In [ ]:

class QuantizedModelPipeline:
    def __init__(self, model_name, quantization_config=None, max_tokens=64):
        self.model_name = model_name
        self.quantization_config = quantization_config
        self.max_tokens = max_tokens
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = None
        self.tokenizer = None

    def load_quantized_model(self):
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name, use_fast=True)
        self.model = AutoModelForCausalLM.from_pretrained(
            self.model_name,
            quantization_config=self.quantization_config,
            device_map="auto",
            torch_dtype=torch.bfloat16
        )
        self.model.eval()
        print(f"Loaded model: {self.model_name} with 4-bit quantization")

    def calculate_perplexity(self, text):
        inputs = self.tokenizer(text, return_tensors="pt", truncation=True, max_length=512).to(self.device)
        with torch.no_grad():
            start_time = time.time()
            outputs = self.model(**inputs, labels=inputs["input_ids"])
            end_time = time.time()
        loss = outputs.loss.item()
        return np.exp(loss), end_time - start_time

    def generate_and_evaluate_bleu(self, input_text, reference_text):
        inputs = self.tokenizer.encode(input_text, return_tensors="pt", truncation=True, max_length=512).to(self.device)

        start_time = time.time()
        outputs = self.model.generate(inputs, max_new_tokens=self.max_tokens)
        end_time = time.time()

        generated_text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)

        smoothie = SmoothingFunction().method4
        reference = [reference_text.split()]
        candidate = generated_text.split()
        bleu_score = sentence_bleu(reference, candidate, smoothing_function=smoothie)

        return generated_text, bleu_score, end_time - start_time

    def evaluate_sample(self, input_text, reference_text):
        ppl, ppl_time = self.calculate_perplexity(input_text)
        generated_text, bleu, gen_time = self.generate_and_evaluate_bleu(input_text, reference_text)

        return {
            "generated": generated_text,
            "perplexity": ppl,
            "perplexity_time": ppl_time,
            "bleu_score": bleu,
            "generation_time": gen_time
        }


In [ ]:

# Define quant config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Create pipeline
pipeline = QuantizedModelPipeline("gpt2", quantization_config=bnb_config)
pipeline.load_quantized_model()

# Load a dataset sample
dataset = load_dataset("xsum", split="validation[:5]")
sample = dataset[0]
input_text = sample["document"]
reference_text = sample["summary"]

# Evaluate
result = pipeline.evaluate_sample(input_text, reference_text)
print(result)


In [ ]:
!git clone https://github.com/ggerganov/llama.cpp
!pip install -r llama.cpp/requirements.txt

In [ ]:
%cd llama.cpp
!make

In [ ]:
%cd .
!git lfs install
!git clone https://huggingface.co/AdamLucek/Orpo-Llama-3.2-1B-15k

In [ ]:
# convert the model to ggml FP16 format
!python3 llama.cpp/convert_hf_to_gguf.py Orpo-Llama-3.2-1B-15k

In [ ]:
# quantize the model to 4-bits (using Q4_K_M method)
!llama.cpp/llama-quantize Orpo-Llama-3.2-1B-15k/Llama-3.2-1B-15k-F16.gguf ./Orpo-Llama-3.2-1B-15k/Orpo-Llama-3.2-1B-15k-Q4_K_M.gguf Q4_K_M